# Tumor Survival Associations Testing Suite

this page tests the functions from the tumor survival associations python notebook

In [1]:
%%capture
%run tumor_survival_associations.ipynb

## Testing Data Imports and Preparation

In [2]:
# survival table that was input from the 'datasets/TCGA_PAAD_survival_data.csv' file
survival

,Cancer_type,submitter_id,PNI,LVI,Filename,Diagnosis Age,Sex,Race Category,Disease Free (Months),Disease Free Status,Overall Survival (Months),Overall Survival Status
0,PAAD,TCGA-HZ-7923,0,0,42bec5f7-7623-42e6-bbdf-514fe3805940.htseq.cou...,65,Male,WHITE,10.32,DiseaseFree,10.32,LIVING
1,PAAD,TCGA-US-A776,0,0,19a3b9bb-d4cb-4925-a87a-57f724141a67.htseq.cou...,61,Male,WHITE,39.95,DiseaseFree,39.95,LIVING
2,PAAD,TCGA-HZ-A77Q,0,0,03630a0c-aa97-4e28-bac9-0206fff669cd.htseq.cou...,55,Female,WHITE,1.08,DiseaseFree,1.08,LIVING
3,PAAD,TCGA-2J-AABA,0,0,c4cadcbb-ae87-43a5-811c-6bedda1d1d8d.htseq.cou...,55,Male,WHITE,15.51,Recurred/Progressed,19.94,DECEASED
4,PAAD,TCGA-F2-6879,0,0,16c63027-f745-41c4-a5e8-f6d9f1fbf1c8.htseq.cou...,57,Male,WHITE,6.01,Recurred/Progressed,10.97,DECEASED
...,...,...,...,...,...,...,...,...,...,...,...,...
149,PAAD,TCGA-2J-AABO,1,-,c642e018-f0cb-4be8-9b19-c944f1daf9cf.htseq.cou...,43,Male,WHITE,14.45,Recurred/Progressed,14.45,LIVING
150,PAAD,TCGA-H6-A45N,1,-,1f997074-0020-47e6-9928-5bf7209c552d.htseq.cou...,88,Female,WHITE,11.93,Recurred/Progressed,13.83,DECEASED
151,PAAD,TCGA-2L-AAQI,1,-,d5612378-33e0-4fe5-ad2f-a1887fb7b5cd.htseq.cou...,66,Male,WHITE,NaN,NaN,3.38,DECEASED
152,PAAD,TCGA-2L-AAQJ,1,-,d7d3fe8e-3885-44e8-934c-50f2a3bbfb2f.htseq.cou...,49,Female,WHITE,NaN,NaN,12.94,DECEASED


In [3]:
# rnaseq table with all of the genes included, imported from 'datasets/Sophia_PAAD_RNAseq.csv'
all_rnaseq

,hgnc_symbol,X00faf8ba.ff90.4214.9d03.6c5e14645d8f.htseq.counts.gz,X0143419f.2abe.4906.bb55.af6010fab05f.htseq.counts.gz,X01f84c45.2058.4e22.b234.52f0a82a97fc.htseq.counts.gz,X03094067.02d4.40c5.b6fa.bb5180dc7eab.htseq.counts.gz,X0349f526.7816.4a7d.9967.1f75dd9ff00a.htseq.counts.gz,X03630a0c.aa97.4e28.bac9.0206fff669cd.htseq.counts.gz,X03761959.a620.440f.bbaa.33bd75afae1c.htseq.counts.gz,X057aa9ac.f22c.4c11.a44d.ad52ae59b4cf.htseq.counts.gz,X05f0ced5.6976.4f43.9be5.fddb3f550adf.htseq.counts.gz,...,f144de50.6126.4912.9c94.824d1eb0fac5.htseq.counts.gz,f2389819.b8fc.460e.821c.01dba313cce1.htseq.counts.gz,f6bd7191.a820.4d86.927a.b4b5f88ebd67.htseq.counts.gz,f748bf78.4dc1.47ad.8611.8186479d3e4b.htseq.counts.gz,f8551a29.d4bd.4954.bf9c.8e10265063de.htseq.counts.gz,f9f63982.b0ee.4cb8.8de5.f885d82137f0.htseq.counts.gz,fb65f821.92cb.402a.ad2f.d4044ca7de4d.htseq.counts.gz,fcd43085.7338.43fe.bc25.9d87b04e227f.htseq.counts.gz,feb22766.4282.47c8.bfe2.7d020b4a15d4.htseq.counts.gz,fef65b57.c58d.4050.8de4.f09f5cd616ce.htseq.counts.gz
0,A1BG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A1BG-AS1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A1CF,4988,3492,3767,2697,4431,3635,5188,4714,3143,...,2424,2757,4434,4202,2321,2500,4916,6074,3980,2784
3,A2M,26,83,113,45,65,34,69,131,74,...,42,70,50,49,75,95,74,108,17,100
4,A2M-AS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37333,ZYG11AP1,27,9,44,8,29,1,1,21,50,...,6,6,10,32,28,82,0,74,8,22
37334,ZYG11B,1799,2857,5383,1757,5474,2362,1575,4241,2492,...,2067,2942,2992,3023,4155,1693,2145,2870,3623,2814
37335,ZYX,1543,1130,1472,914,559,4893,13436,1859,2283,...,2489,741,2192,1739,3457,1376,2757,2559,1388,1098
37336,ZYXP1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


1. test create counts list checks that all of the filenames from the 'Filename' column have been added to the counts list

In [4]:
def test_create_counts_list(patient_table):
    counts = create_counts_list(patient_table)
    return len(counts) == patient_table['Filename'].count()

test_create_counts_list(survival)

True

2. test filter patients checks to see if the column list contains elements from create counts list

weird seems like there's an X in front of the column names, but not on all of them

In [7]:
def test_filter_patients(rnaseq_table, patient_table):
    counts = create_counts_list(patient_table)
    column_list = rnaseq_table.columns.values
    return counts, column_list#all(item in column_list for item in counts)

test_filter_patients(rnaseq_survival, survival)

(['42bec5f7.7623.42e6.bbdf.514fe3805940.htseq.counts.gz',
  '19a3b9bb.d4cb.4925.a87a.57f724141a67.htseq.counts.gz',
  '03630a0c.aa97.4e28.bac9.0206fff669cd.htseq.counts.gz',
  'c4cadcbb.ae87.43a5.811c.6bedda1d1d8d.htseq.counts.gz',
  '16c63027.f745.41c4.a5e8.f6d9f1fbf1c8.htseq.counts.gz',
  '08b04038.cfb4.4a4c.9e20.1585a4af6826.htseq.counts.gz',
  '2e33a2c5.2196.49c4.866a.34b12d984c15.htseq.counts.gz',
  '8a799dfa.c1b5.4b13.9c91.6cbfe2abbc9f.htseq.counts.gz',
  '7cccb817.cdff.45c0.a4cd.9de57d7d8c48.htseq.counts.gz',
  '4b377b21.fed6.4684.bd4c.93850e8b5b8a.htseq.counts.gz',
  '7186ee30.db1a.43ad.955b.886cd5b5e771.htseq.counts.gz',
  'e7cc80ef.4b87.47d9.bebe.1fb05b5b04a2.htseq.counts.gz',
  '4929062b.3127.4038.8313.c20cbd274be4.htseq.counts.gz',
  '65f1f44c.1f60.45cd.8e1d.16304536c4d5.htseq.counts.gz',
  '057aa9ac.f22c.4c11.a44d.ad52ae59b4cf.htseq.counts.gz',
  'e1e6b7e8.884b.4074.93f7.1feba917f805.htseq.counts.gz',
  'b244f324.fd8a.4d4b.b8f5.bad973c649d5.htseq.counts.gz',
  '0be94b2f.fc

3. test filter by submitter id 1 checks how many submitter ids we have

4. test filter by submitter id 2 checks that every item in submitter id list is present in the paad patient table and vice versa

5. test filter by submitter id 3 checks that intersection and difference take the opposites - intersection selects the columns from the list, difference selects the columns not from the list

In [8]:
def test_filter_by_submitter_id_1(rnaseq_table, patient_table):
    submitter_id_list = patient_table['submitter_id'].tolist()
    return len(submitter_id_list)

test_filter_by_submitter_id_1(all_rnaseq, survival)

# 154 is consistent with the length of the survival table

154

In [10]:
def test_filter_by_submitter_id_2(rnaseq_table, patient_table):
    submitter_id_list = patient_table['submitter_id'].tolist()
    all_paad_patients = pd.read_csv('datasets/Sophia_PAAD_patients.csv', delimiter = ',')
    paad_patients = all_paad_patients[all_paad_patients['submitter_id'].isin(submitter_id_list)]
    
    return (all(item in submitter_id_list for item in paad_patients['submitter_id'].tolist()) and
            all(item in paad_patients['submitter_id'].tolist() for item in submitter_id_list))

test_filter_by_submitter_id_2(all_rnaseq, survival)

True

In [17]:
def test_filter_by_submitter_id_3(rnaseq_table, patient_table):
    submitter_id_list = patient_table['submitter_id'].tolist()
    all_paad_patients = pd.read_csv('datasets/Sophia_PAAD_patients.csv', delimiter = ',')
    paad_patients = all_paad_patients[all_paad_patients['submitter_id'].isin(submitter_id_list)]
    
    counts = create_counts_list(paad_patients)
    
    selected = len(all_rnaseq[all_rnaseq.columns.intersection(counts)].columns.values)
    not_selected = len(all_rnaseq[all_rnaseq.columns.difference(counts)].columns.values)
    return len(all_rnaseq.columns.values) - selected - not_selected
    
test_filter_by_submitter_id_3(all_rnaseq, survival)

0